In [79]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

import pandas as pd 
import numpy as np
import csv
import re
from re import sub

from typing import Pattern
import sys
from unidecode import unidecode

In [80]:
path_init = "data/_57793TransmissionsSurvivors_2014.csv"
path_new_sentiments = "data/data_train.csv"

PATTERN_SUPP_HTML : Pattern = re.compile(r'<(.*?)>')
REPLACE_NO_SPACE : Pattern= re.compile(r"[.;:!=\%?,\"()\\]")
REPLACE_WITH_SPACE : Pattern = re.compile(r"(<br\s*/><br\s*/>)|(\-)|(\/)|°")

In [87]:
# nettoyer le texte 
def preprocess_clean(reviews):
    reviews = reviews.lower() # mettre le texte en minuscule

    reviews = re.sub(r'\bagrave\b', 'a', reviews) #replacer agrave par a (!à)
    reviews = re.sub(r'\beacut\b|\begrave\b|\bpreacute\b', 'e', reviews)#

    sub_html = re.findall(PATTERN_SUPP_HTML, reviews) #supprimer les balise html
    for html in sub_html:
        if html:
            reviews = re.sub(r'\s+', ' ', reviews)
            reviews = re.sub(r"\'", ' ', reviews)
            reviews = reviews.replace(html, ' ')
            reviews = reviews.replace('< >', ' ')

    reviews = REPLACE_NO_SPACE.sub(' ', reviews) #supprimer tout les caracteres [.;:!=\%?,\"()\[\]]
    reviews = REPLACE_WITH_SPACE.sub(' ', reviews) #supprimer espace entre balise html au cas où y'en a 
    reviews = reviews.replace('ttt', 'traitement') 
    reviews = re.sub(r"([+])", ' plus ', reviews)
    reviews = re.sub(r"([+])", ' ', reviews)
    #reviews = re.sub(r'[0-9]', ' ', reviews)
    
    #séparer les chiffres des lettres 1923example1212 = 1923 example1212
    reviews = re.sub(r'([^0-9])(\d{1,2})', r'\1 \2', reviews)
    #séparer les chiffres des lettres 1923example1212 = 1923example 1212 résultat final 1923 example 1212 
    reviews = re.sub(r'(\d{1,2})([^0-9])', r'\1 \2', reviews)
    #reviews = re.sub(r'\b\w{1,3}\b',' ', reviews)

    
    #supprimer tout espace de plus qui peut exister à la fin de traitement
    #print(reviews)
    return reviews

In [88]:
def clean_text_to_word_list(text, remove_polish_letters):
    text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub('(\d)([a-zA-Z]+)', r'\1 \2', text)
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r'([^0-9])(\d{1,2})', r'\1 \2', text)
    text = sub(r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', " ", text)
    text = sub(r"\s{2,}", " ", text)
    #text = text.split()

    return text

In [89]:
df = pd.read_csv('../2-data/NETREAD.NETSOINS_TRANSMISSIONS.csv', encoding='utf-8', sep=";")
df.head()

,id_resident,id_etablissement,date_T,description,label
0,42973,557,1418663163,"Vue ce jour poursuite du suivi, Mme Cautain es...",NaN
1,25134,739,1416909807,ai posé la question au dr touggani concernant ...,NaN
2,28144,561,1418664063,tres agrssive en ce moment. surveillance sinon...,NaN
3,16416,650,1421399533,reeducation de membre superieur gauche et de l...,NaN
4,10272,670,1418085149,Elle n'arrète pas de m'appeler sur le téléhone...,NaN


In [96]:
df = df.head(1000)
df_2 = df[['description']] #df_2.columns = ['new']
df_2[df_2['description'].str.strip().astype(bool)]
#df_2.description = df_2.description.apply(lambda x: clean_text_to_word_list(x, unidecode))
df_2.description = df_2.description.apply(lambda x: preprocess_clean(x))
df_2 = df_2[~df_2.description.str.contains(r'\bsuite\b|\berreur\b')]
df_2 = df_2.sample(frac=1).reset_index(drop=True)
df_2.head(20)



,description
0,8 h ta 15 9 et pouls 86
1,attention ne plus mettre d'alloplastine au niv...
2,va bcp mieux t 37 5 ta 13 7 sat 97 sous oxygen...
3,tousse beaucoup depuis hier fesses trés rouge ...
4,résultats du bilan sang de ce matin perturbé
5,cet am semblait perdu au moment du gouter allo...
6,toilette faite par as ce matin dès 7 heures
7,ide prèvenu
8,sacrum abîmé
9,creation evaluation evaluation norton réalisé...


In [101]:
df_2.to_csv('data-clean.csv', sep=';', encoding='utf-8', index_label='Index') #

In [75]:
d12f = pd.DataFrame(dict(A=[5,3,5,6], C=["foo","bar","foo suite XYZ bar", "bat"]))
d12f.head()

,A,C
0,5,foo
1,3,bar
2,5,foo suite XYZ bar
3,6,bat


In [76]:
d12f[~d12f.C.str.contains(r"\bsuite\b|\berreur\b")]

,A,C
0,5,foo
1,3,bar
3,6,bat
